In [1]:
import csv
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

In [15]:
!ls /home/ljb/Downloads/*.csv

/home/ljb/Downloads/job_27469ef0-e146-4aac-a655-575a1013fe02.csv
/home/ljb/Downloads/job_3328ed71-e01b-47c4-a2a9-be6276a59f08.csv
/home/ljb/Downloads/job_3c081b1c-2215-42a1-bd7f-6e7395f087ad.csv
/home/ljb/Downloads/job_71913fe0-d7bd-4740-a771-8afcd1356415.csv
/home/ljb/Downloads/job_98d07a69-caba-4315-b14d-490ad5c2759a.csv
/home/ljb/Downloads/job_eeaf7eac-110e-4594-97ad-c4659229169e.csv
/home/ljb/Downloads/job_fd760c83-3db1-4595-b6e2-7ad2a02cca52.csv


In [16]:
results = "/home/ljb/Downloads/job_eeaf7eac-110e-4594-97ad-c4659229169e.csv"  # turtle_gridsearch # bad names

In [17]:
df = pd.read_csv(results)
df.tail()

,scenario_id,scenario_name,metric:error_orientation_final_deg,metric:cummulated_distance_final_m,metric:cumulative_distance_final_m,metric:error_horizontal_final_m,metric:cumulated_distance_final_m,metric:time_delta_max_ms
43,5_turtle_gridsearch_odom,turtle_gridsearch_odom,13.876281,20.40,NaN,1.629315,NaN,0.455618
44,6_turtle_gridsearch_odom,turtle_gridsearch_odom,38.027365,20.48,NaN,4.837369,NaN,0.411749
45,7_turtle_gridsearch_odom,turtle_gridsearch_odom,37.376912,20.40,NaN,4.830069,NaN,0.498056
46,8_turtle_gridsearch_odom,turtle_gridsearch_odom,0.923899,20.40,NaN,0.240754,NaN,0.649214
47,9_turtle_gridsearch_odom,turtle_gridsearch_odom,1.110862,20.40,NaN,0.189230,NaN,0.514746


In [30]:
df.columns[[3, 4, 6]]

Index(['metric:cummulated_distance_final_m',
       'metric:cumulative_distance_final_m',
       'metric:cumulated_distance_final_m'],
      dtype='object')

In [27]:
df['temp'] = df[df.columns[[3, 4, 6]]].apply(
    lambda x: x.dropna()[0], axis=1
)

In [29]:
df['metric:cumulative_distance_final_m'] = df['temp']

In [36]:
df = df.drop(columns=df.columns[[3, 4, 6]], axis=1)

In [43]:
df.tail()

,scenario_id,scenario_name,metric:error_orientation_final_deg,metric:error_horizontal_final_m,metric:time_delta_max_ms,metric:cumulative_distance_final_m
43,5_turtle_gridsearch_odom,turtle_gridsearch_odom,13.876281,1.629315,0.455618,20.40
44,6_turtle_gridsearch_odom,turtle_gridsearch_odom,38.027365,4.837369,0.411749,20.48
45,7_turtle_gridsearch_odom,turtle_gridsearch_odom,37.376912,4.830069,0.498056,20.40
46,8_turtle_gridsearch_odom,turtle_gridsearch_odom,0.923899,0.240754,0.649214,20.40
47,9_turtle_gridsearch_odom,turtle_gridsearch_odom,1.110862,0.189230,0.514746,20.40


In [42]:
df.rename(columns={'temp': 'metric:cumulative_distance_final_m'}, inplace=True)

In [44]:
# assumptions: 
# each metric name starts with 'metric:'
# each metric name ends with underscore + the metric's unit: '_deg'
metrics = sorted([m for m in df.columns if 'metric' in m])    # metric:time_delta_max_ms
metric_names = [m.split("metric:")[-1] for m in metrics]      # time_delta_max_ms
metric_units = [f"{n.split('_')[-1]}" for n in metric_names]  # ms
metric_titles = [f"{' '.join(n.split('_')[:-1])} ({u})" for n, u in zip(metric_names, metric_units)]  # time delta max (ms)
metric_stds = [np.std(df[m]) for m in metrics]

fig = make_subplots(
    rows=1, 
    cols=len(metrics),
    subplot_titles=[f"{title} <br> std = {std:.1g} {unit}" for title, std, unit in zip(metric_titles, metric_stds, metric_units)]
)
for i, metric in enumerate(metrics):
    fig.add_trace(
        px.box(
            df, 
            y=metric, 
            points="all",
        )['data'][0],
        row=1, col=i+1
    )
fig.update_layout(template='presentation', title="Basic statistics for each metric")
fig

In [18]:
fig

In [10]:
fig

In [68]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(df.index), y=df[metrics[1]], name=metric_titles[1], mode='markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=list(df.index), y=df[metrics[2]], name=metric_titles[2], mode='markers'),
    secondary_y=True,
)
fig.update_xaxes(title_text="scenario result index")
fig.update_yaxes(title_text=metric_units[1], secondary_y=False)
fig.update_yaxes(title_text=metric_units[2], secondary_y=True)

fig.update_layout(template='presentation', title_text="Metrics Comparison For Each Scenario of the Grid Search")
fig